# 2 задача
## Найти решение краевой задачи для одномерного стационарного уравнения теплопроводности

### Постановка задачи (вариант №1 задание №4)

#### Уравнение теплопроводности:

\begin{gather*}
   {d \over dx} [k(x){du \over dx}] - q(x)u = -f(x)\\
    k(x) = sin^2(x) + 1;\; \; \;
    q(x) = sin(x); \; \; \;
    f(x) = e^x
\end{gather*}

\begin{equation}
0 \leq x \leq 1 
\end{equation}

#### Краевые условия задачи:

\begin{equation}
\left.
  \begin{array}{ccc}
      k(0) u_x'(0) = u(0) - 1 \\
      -k(1) u_x'(1) = u(1) - 1
  \end{array}
\right\}
\end{equation}

#### Модельная задача и её решение:

Примем коэффициенты в уравнении константами:

\begin{gather*}
    K = k(0.5) = sin^2(0.5) + 1;\; \; \;
    Q = q(0.5) = sin(0.5); \; \; \;
    F = f(0.5) = e^{0.5}
\end{gather*}

Тем самым модельная задача станет такой:

\begin{gather*}
   K{d^2u \over dx^2} - Qu = -F\\
\end{gather*}

\begin{equation}
0 \leq x \leq 1 
\end{equation}

Ее решение:
\begin{equation}
u(x) = u_{общ}(x) + u_{частн}(x), \;\;\;где
\end{equation}

\begin{equation}
u_{общ}(x) = C_1 e^{\sqrt{Q \over K}x} + C_2 e^{-\sqrt{Q \over K}x} 
\end{equation}

\begin{equation}
u_{частн}(x) = {F \over Q}
\end{equation}

Константы - из краевых условий:

\begin{equation}
\left.
  \begin{array}{ccc}
      K ({\sqrt{Q \over K}C_1}-{\sqrt{Q \over K}C_2}) = {F \over Q} + C_1 + C_2 - 1 \\
      -K (e^{\sqrt{Q \over K}}{\sqrt{Q \over K}C_1} - e^{-\sqrt{Q \over K}}{\sqrt{Q \over K}C_2})= {F \over Q} + e^{\sqrt{Q \over K}}{\sqrt{Q \over K}}C_1 + e^{-\sqrt{Q \over K}}{\sqrt{Q \over K}}C_2 - 1
  \end{array}
\right\}
\end{equation}


In [5]:
u0, u1 = 0, 1            # концы отрезка
L = 10241                 # число узлов
h = (u1 - u0) / (L - 1)  # шаг сетки

delta1, delta2 = 1, 1
eps1, eps2 = 1, 1

### Заменим производные в узлах сетки конечными разностями

\begin{equation}
\left\{
  \begin{array}{ccc}
      \frac{k_{l+1/2} \: (u_{l+1}-u_l) - k_{l-1/2} \: (u_l-u_{l-1})}{h^2} + q_{l} u_l = -f_l, \;\;\;\;\; l \in [1;L-1]\\
      k_0 {u_1-u_0 \over h} = \delta_1 u_0 - \varepsilon_1 \\ 
      -k_L {u_L-u_{L-1} \over h} = \delta_2 u_L - \varepsilon_2
  \end{array}
\right.
\end{equation}

#### Теперь необходимо решить полученную систему линейных уравнений

Перепишем систему в более удобном виде:

\begin{equation} 
\left\{
  \begin{array}{ccc}
      k_{l-1/2} \: u_{l-1} + (h^2 q_l - k_{l-1/2} - k_{l+1/2}) \: u_l + k_{l+1/2} \: u_{l+1} = -h^2 f_l, \;\;\;\;\; l \in [1;L-1]\\
      (k_0 + h \delta_1)\:u_0 - k_0 u_1  = h \varepsilon_1 \\ 
      -k_L u_{L-1} + (k_L + h \delta_2)\:u_L = h \varepsilon_2\\
  \end{array}
\right.
\end{equation}

Здесь разумно сделать замену переменных в уравнениях системы:

\begin{align*}
a_0& = 0           &  b_0 & = k_0 + h \delta_1                 &  c_0& = -k_0          &   d_0 & = h \varepsilon_1\\
a_l& = k_{l-1/2}   &  b_l&  = h^2 q_l - k_{l-1/2} - k_{l+1/2}  &  c_l& = k_{l+1/2}     &   d_l & = -h^2 f_l \\
a_L& = -k_L        &  b_L&  =k _L + h \delta_2                 &  c_L& = 0             &   d_L & = h \varepsilon_2
\end{align*}

Таким образом, система алгебраических уравнений примет вид:

\begin{equation} 
a_l u_{l-1} + b_l u_l + c_l u_{l+1} = d_l, \; \; \; l \in [0, L]
\end{equation}

Это матрица трёхдиагонального вида. Для таких матриц используется метод прогонки (алгоритм Томаса). Мы же воспользуемся встроенной функцией из numpy.linalg для разреженной матрицы из scipy.sparse

In [6]:
from math import *
from scipy.sparse import dia_matrix
#from numpy import linalg as LA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

k = lambda x: sin(x)**2 + 1
q = lambda x: sin(x)
f = lambda x: exp(x)

a0, b0, c0, d0 = 0, k(0) + h*delta1, k(0), h*eps1
aL, bL, cL, dL = -k(1), k(1) + h*delta2, 0, h*eps2

a = lambda x: k(x - h/2)
b = lambda x: h**2 * q(x) - k(x - h/2) - k(x + h/2)
c = lambda x: k(x + h/2)
d = lambda x: -h**2 * f(x)

In [48]:
upper_arr = [0, c0] + [c(l*h) for l in range(1, L-1)] 
middle_arr = [b0] + [b(l*h) for l in range(1, L-1)] + [bL]
lower_arr = [a(l*h) for l in range(1, L-1)] + [aL, 0]

# Создадим матрицу системы:
A = dia_matrix((np.array([lower_arr, middle_arr, upper_arr]), [-1, 0, 1]), shape=(L, L)).toarray()

# Создадим вектор свободных членов:
B = np.array([d0] + [d(l*h) for l in range(1, L-1)] + [dL])

# Получим решение:
solution = np.linalg.solve(A, B) 

In [51]:
sol32 = solution

In [57]:
sol64 = solution

In [62]:
sol128 = solution

In [67]:
sol256 = solution

In [70]:
sol512 = solution

In [75]:
sol1024 = solution

In [80]:
sol2048 = solution

In [46]:
# решения в 11 точках на [0;1]
pd.DataFrame([sol32,sol64,sol128,sol256,sol512,sol1024,sol2048],columns=(np.linspace(0,1,num=11)),index=[321,641,1281,2561,5121,10241,20481])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
321,1.479940,1.519472,1.553606,1.576407,1.591069,1.596251,1.592876,1.586196,1.570117,1.550691,1.522534
641,1.477129,1.520079,1.552093,1.574917,1.588582,1.593881,1.592105,1.584172,1.569240,1.548461,1.521228
1281,1.477167,1.519399,1.551838,1.574576,1.588313,1.593617,1.591992,1.583452,1.568701,1.548059,1.520951
2561,1.476720,1.519310,1.551794,1.574515,1.588133,1.593603,1.591752,1.583252,1.568505,1.547641,1.520470
5121,1.476697,1.519310,1.551797,1.574497,1.588131,1.593606,1.591752,1.583242,1.568499,1.547640,1.520474
10241,1.476695,1.519310,1.551793,1.574491,1.588134,1.593600,1.591745,1.583237,1.568495,1.547637,1.520466
20481,1.476695,1.519310,1.551792,1.574491,1.588132,1.593599,1.591743,1.583237,1.568494,1.547635,1.520466


In [47]:
err32 = abs(sol32-sol2048)/sol2048
err64 = abs(sol64-sol2048)/sol2048
err128 = abs(sol128-sol2048)/sol2048
err256 = abs(sol256-sol2048)/sol2048
err512 = abs(sol512-sol2048)/sol2048
err1024 = abs(sol1024-sol2048)/sol2048

In [48]:
# относительная ошибка 
pd.DataFrame([err32,err64,err128,err256,err512,err1024],columns=(np.linspace(0,1,num=11)),index=[321,641,1281,2561,5121,10241])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
321,2.197300e-03,1.069882e-04,0.001169,1.216844e-03,1.849765e-03,1.663874e-03,0.000712,1.868954e-03,1.034455e-03,0.001974,1.360406e-03
641,2.936200e-04,5.066230e-04,0.000194,2.703216e-04,2.835046e-04,1.768081e-04,0.000227,5.905102e-04,4.757232e-04,0.000533,5.014677e-04
1281,3.199490e-04,5.878724e-05,0.000030,5.384604e-05,1.142247e-04,1.091851e-05,0.000156,1.362249e-04,1.316321e-04,0.000274,3.194838e-04
2561,1.710826e-05,2.031554e-07,0.000001,1.530694e-05,1.145093e-06,2.345754e-06,0.000005,9.753867e-06,6.609002e-06,0.000004,2.970939e-06
5121,1.112332e-06,9.212310e-08,0.000003,3.934295e-06,1.437013e-07,4.165277e-06,0.000006,3.469799e-06,2.908202e-06,0.000003,5.426553e-06
10241,2.150555e-07,1.964875e-07,0.000001,1.737105e-07,1.384587e-06,3.438589e-07,0.000001,1.260307e-07,6.619054e-07,0.000001,4.543082e-08
